# Import dependencies

In [ ]:
import pandas as pd
import numpy as np


# Import File

### Update your file location here

In [ ]:
df = pd.read_csv("/AgenticAI/customer_support_tickets.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

In [ ]:
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [ ]:
# Gather descriptive statistics about the data
df.describe()

,Ticket ID,Customer Age,Customer Satisfaction Rating
count,8469.000000,8469.000000,2769.000000
mean,4235.000000,44.026804,2.991333
std,2444.934048,15.296112,1.407016
min,1.000000,18.000000,1.000000
25%,2118.000000,31.000000,2.000000
50%,4235.000000,44.000000,3.000000
75%,6352.000000,57.000000,4.000000
max,8469.000000,70.000000,5.000000


In [ ]:
# Display all column names
df.columns

Index(['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age',
       'Customer Gender', 'Product Purchased', 'Date of Purchase',
       'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating'],
      dtype='object')

In [ ]:
# Check for missing values
print(df.isnull().sum())

Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64


# Observations & Actions

For RAG enrichment for our support agent scenario, we want columns that give the LLM extra context without leaking PII (Personally Identifiable Information) and without bloating the embedding vectors with irrelevant metadata.


**Columns to Keep for RAG**


| Column                 | Why Keep                                                                       | How to Use                                               |
| ---------------------- | ------------------------------------------------------------------------------ | -------------------------------------------------------- |
| **Ticket ID**          | Good for traceability and debugging search results.                            | Store as metadata in OpenSearch (not in embedding text). |
| **Product Purchased**  | Crucial for context — lets retrieval filter by product type/model.             | Store both as metadata and inline in embedding text.     |
| **Ticket Subject**     | Usually short, keyword-rich description — good for search.                     | Include in embedding text.                               |
| **Ticket Description** | The actual customer problem — essential for retrieval.                         | Include in embedding text.                               |
| **Resolution**         | Past solution — very valuable for RAG answer quality.                          | Include in embedding text so LLM can reuse/adapt.        |
| **Ticket Type**        | Helps retrieval match similar categories (billing, technical, warranty, etc.). | Store as metadata for filtering.                         |
| **Ticket Priority**    | Can help prioritize retrieval results if urgent tickets are more relevant.     | Store as metadata (optional for filtering).              |
| **Ticket Channel**     | Sometimes useful if certain channels produce different wording/styles.         | Store as metadata (optional).                            |




---



**Columns to Remove (PII or low value for RAG)**

| Column                       | Reason to Remove                                                                                                            |
| ---------------------------- | --------------------------------------------------------------------------------------------------------------------------- |
| Customer Name                | PII, not useful for retrieval.                                                                                              |
| Customer Email               | PII, must remove for compliance.                                                                                            |
| Customer Age                 | Probably not relevant for product troubleshooting in your case.                                                             |
| Customer Gender              | Risk of bias, likely irrelevant to the resolution.                                                                          |
| Date of Purchase             | Could be relevant only if troubleshooting depends on purchase date (e.g., warranty). Otherwise, keep as metadata if needed. |
| First Response Time          | Operational metric, not useful for answering queries.                                                                       |
| Time to Resolution           | Same as above.                                                                                                              |
| Customer Satisfaction Rating | Only useful for analytics, not RAG context.                                                                                 |


## Data Cleaning & Restructuring

### Remove unwanted Columns

In [ ]:
df_refined = df.drop(columns=['Customer Name', 'Customer Email', 'Customer Age', 'Customer Gender', 'Date of Purchase', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating'])

In [ ]:
df_refined.head()

,Ticket ID,Product Purchased,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel
0,1,GoPro Hero,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media
1,2,LG Smart TV,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat
2,3,Dell XPS,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media
3,4,Microsoft Office,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media
4,5,Autodesk AutoCAD,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email


### Rename Columns

In [ ]:
### Rename columns as needed
df_refined = df_refined.rename(columns={'Ticket ID': 'ticket_id',
                        'Product Purchased': 'product_purchased',
                        'Ticket Type':'type',
                        'Ticket Subject': 'subject',
                        'Ticket Description': 'description',
                        'Ticket Status': 'status',
                        'Resolution': 'resolution',
                        'Ticket Priority': 'priority',
                        'Ticket Channel': 'channel'
                       }).copy()

# Display all column names after renaming
df_refined.columns

Index(['ticket_id', 'product_purchased', 'type', 'subject', 'description',
       'status', 'resolution', 'priority', 'channel'],
      dtype='object')

In [ ]:
df_refined.head()

,ticket_id,product_purchased,type,subject,description,status,resolution,priority,channel
0,1,GoPro Hero,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media
1,2,LG Smart TV,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat
2,3,Dell XPS,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media
3,4,Microsoft Office,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media
4,5,Autodesk AutoCAD,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email


### Replace {product_purchased} variable with actual product names

In [ ]:
df_refined["description"] = df_refined.apply(lambda row: row["description"].replace("{product_purchased}", str(row["product_purchased"])), axis=1)

In [ ]:
df_refined.head()

,ticket_id,product_purchased,type,subject,description,status,resolution,priority,channel
0,1,GoPro Hero,Technical issue,Product setup,I'm having an issue with the GoPro Hero. Pleas...,Pending Customer Response,NaN,Critical,Social media
1,2,LG Smart TV,Technical issue,Peripheral compatibility,I'm having an issue with the LG Smart TV. Plea...,Pending Customer Response,NaN,Critical,Chat
2,3,Dell XPS,Technical issue,Network problem,I'm facing a problem with my Dell XPS. The Del...,Closed,Case maybe show recently my computer follow.,Low,Social media
3,4,Microsoft Office,Billing inquiry,Account access,I'm having an issue with the Microsoft Office....,Closed,Try capital clearly never color toward story.,Low,Social media
4,5,Autodesk AutoCAD,Billing inquiry,Data loss,I'm having an issue with the Autodesk AutoCAD....,Closed,West decision evidence bit.,Low,Email


### Filter Records

As this is just a POC project, hence to keep it simple I will fetch and enrich RAG only for one company (Eg. Canon) product.

In [ ]:
df_final = df_refined[df_refined['product_purchased'].str.startswith('Canon', na=False)]

In [ ]:
df_final.head()

,ticket_id,product_purchased,type,subject,description,status,resolution,priority,channel
19,20,Canon EOS,Refund request,Software bug,I'm having an issue with the Canon EOS. Please...,Closed,Wish mouth build resource though.,Low,Chat
36,37,Canon EOS,Refund request,Hardware issue,I'm having an issue with the Canon EOS. Please...,Pending Customer Response,NaN,Medium,Email
69,70,Canon DSLR Camera,Refund request,Refund request,I'm unable to access my Canon DSLR Camera acco...,Closed,Answer story series imagine discover.,High,Chat
85,86,Canon EOS,Product inquiry,Peripheral compatibility,I've forgotten my password for my Canon EOS ac...,Open,NaN,High,Social media
143,144,Canon DSLR Camera,Cancellation request,Account access,I'm having an issue with the Canon DSLR Camera...,Open,NaN,Critical,Email


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 446 entries, 19 to 8459
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ticket_id          446 non-null    int64 
 1   product_purchased  446 non-null    object
 2   type               446 non-null    object
 3   subject            446 non-null    object
 4   description        446 non-null    object
 5   status             446 non-null    object
 6   resolution         164 non-null    object
 7   priority           446 non-null    object
 8   channel            446 non-null    object
dtypes: int64(1), object(8)
memory usage: 34.8+ KB


In [ ]:
df_final.to_csv('customer_tickets.csv', index=False)

In [ ]:
df_support = pd.read_csv("/content/customer_support_tickets.csv")

In [ ]:
df_support.head()

,ticket_id,product_purchased,type,subject,description,status,resolution,priority,channel
0,20,Canon EOS,Refund request,Software bug,I'm having an issue with the {product_purchase...,Closed,Identified software bug in Canon EOS firmware....,Low,Chat
1,70,Canon DSLR Camera,Refund request,Refund request,I'm unable to access my {product_purchased} ac...,Closed,Assisted with account recovery by resetting cr...,High,Chat
2,200,Canon EOS,Product inquiry,Software bug,I'm having an issue with the {product_purchase...,Closed,Confirmed firmware update caused UI glitch. Ro...,Medium,Chat
3,220,Canon EOS,Technical issue,Refund request,I'm having an issue with the {product_purchase...,Closed,Guided user through menu navigation to locate ...,Critical,Social media
4,323,Canon DSLR Camera,Refund request,Battery life,I'm having an issue with the {product_purchase...,Closed,Diagnosed battery drain due to app cache. Reco...,Critical,Social media


In [ ]:
# Merge df_final with df_support on 'ticket_id' to get the updated resolution column
df_final = df_final.merge(df_support[['ticket_id', 'resolution']], on='ticket_id', how='left', suffixes=('_original', '_updated'))

# Replace the original resolution column with the updated one
df_final['resolution'] = df_final['resolution_updated']

# Drop the temporary columns created during the merge
df_final = df_final.drop(columns=['resolution_original', 'resolution_updated'])

# Display the head of the updated dataframe
df_final.head()

,ticket_id,product_purchased,type,subject,description,status,priority,channel,resolution
0,20,Canon EOS,Refund request,Software bug,I'm having an issue with the Canon EOS. Please...,Closed,Low,Chat,Identified software bug in Canon EOS firmware....
1,37,Canon EOS,Refund request,Hardware issue,I'm having an issue with the Canon EOS. Please...,Pending Customer Response,Medium,Email,NaN
2,70,Canon DSLR Camera,Refund request,Refund request,I'm unable to access my Canon DSLR Camera acco...,Closed,High,Chat,Assisted with account recovery by resetting cr...
3,86,Canon EOS,Product inquiry,Peripheral compatibility,I've forgotten my password for my Canon EOS ac...,Open,High,Social media,NaN
4,144,Canon DSLR Camera,Cancellation request,Account access,I'm having an issue with the Canon DSLR Camera...,Open,Critical,Email,NaN


In [ ]:
df_final.to_csv('customer_tickets_merged.csv', index=False)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ticket_id          475 non-null    int64 
 1   product_purchased  475 non-null    object
 2   type               475 non-null    object
 3   subject            475 non-null    object
 4   description        475 non-null    object
 5   status             475 non-null    object
 6   priority           475 non-null    object
 7   channel            475 non-null    object
 8   resolution         182 non-null    object
dtypes: int64(1), object(8)
memory usage: 33.5+ KB
